In [109]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import sqlite3

conn = sqlite3.connect('smartAggie.sqlite')

In [110]:
# Professors vs comment quality rating

professors_df = pd.read_sql_query('SELECT Name, Department, `Prof Quality Rating`, `Level of Difficulty`, `Would Take Again`, `Total Ratings` from Professors', con=conn)

In [111]:
# Mild correlation between Would Take Again and Prof Quality Rating
professors_df['Would Take Again'] = [abs(number) for number in professors_df['Would Take Again']]
corr_df = professors_df.drop(columns=['Total Ratings']).corr(method='spearman', numeric_only= True)

fig = px.imshow(corr_df, labels=dict(color='Correlation'), x=corr_df.columns, y=corr_df.columns, color_continuous_scale='teal', text_auto=True, title='Professors correlation matrix')
fig.show()

In [112]:
professors_df['Would Take Again'] = professors_df['Would Take Again']/100.0
professors_df['Level of Difficulty'] = professors_df['Level of Difficulty']/5
professors_df['Prof Quality Rating'] = professors_df['Prof Quality Rating']/5
fig = px.scatter_ternary(professors_df, a='Level of Difficulty', b='Prof Quality Rating', c='Would Take Again', color='Department', size='Total Ratings')
fig.show()

In [113]:
filtered_prof_df = professors_df[professors_df['Total Ratings'] > 100]
fig = px.scatter(filtered_prof_df, x='Name', y='Prof Quality Rating', color='Department', size='Total Ratings')
fig.show()

In [114]:
prof_dict = {}
deps_dict = {}

for index, rows in professors_df.iterrows():
    if rows['Department'] not in prof_dict:
        prof_dict[rows['Department']] = []
        deps_dict[rows['Department']] = 0
    prof_dict[rows['Department']].append((int(rows['Total Ratings']), float(rows['Prof Quality Rating']), rows['Name']))
    deps_dict[rows['Department']] += int(rows['Total Ratings'])

top_prof_df = pd.DataFrame(columns=['Department', 'Professor', 'Rating'])

deps_dict = dict(sorted(deps_dict.items(), key=lambda item: item[1], reverse=True))

for dep in list(deps_dict.keys())[:5]:
    profs = sorted(prof_dict[dep], reverse=True)[:5]
    # if len(profs) > 5:
    #     profs = profs[:5]
    for _, rating, prof in profs:
        top_prof_df.loc[len(top_prof_df)] = [dep, prof, rating]
    
# fig = px.histogram(top_prof_df, x='Professor', y='Rating', color='Department', text_auto=True)
# fig.show()

courses = pd.read_sql_query('SELECT ProfName, `Subject Name`, `Course Code`, Quarter from Courses', con=conn)
courses['Quarter'] = courses['Quarter'].astype(str)
prof_course_df = pd.merge(top_prof_df, courses, left_on=['Department', 'Professor'], right_on=['Subject Name', 'ProfName'], how='left')

# fig = px.bar(prof_course_df, x='Course Code', y='Rating', color='Quarter', text_auto=True, barmode='group')
fig = px.parallel_categories(prof_course_df, dimensions=['Department', 'Quarter', 'Course Code'], color='Rating')
fig.show()

In [115]:
filtered_prof_diff_df = professors_df[professors_df['Total Ratings'] > 100]
fig = px.violin(filtered_prof_diff_df, y='Level of Difficulty', color='Department', violinmode='overlay')
fig.show()

In [122]:
diff_dict = {}
diff_prof_df = pd.DataFrame(columns=['Department', 'Professor', 'Difficulty'])
for index, rows in professors_df.iterrows():
    if rows['Department'] in list(deps_dict.keys())[:5]:
        if rows['Department'] not in diff_dict:
            diff_dict[rows['Department']] = []
        diff_dict[rows['Department']].append((rows['Total Ratings'], rows['Level of Difficulty'], rows['Name']))

for dep in list(deps_dict.keys())[:5]:
    diff = sorted(diff_dict[dep], reverse=True)[:5]
    # if len(profs) > 5:
    #     profs = profs[:5]
    for _, rating, prof in diff:
        diff_prof_df.loc[len(diff_prof_df)] = [dep, prof, rating]    

prof_course_diff_df = pd.merge(courses, diff_prof_df, right_on=['Department', 'Professor'], left_on=['Subject Name', 'ProfName'], how='inner')
fig = px.sunburst(prof_course_diff_df, path=['Department', 'Professor', 'Quarter', 'Course Code'], values='Difficulty')
fig.show()